In [ ]:
%cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
import os
import numpy as np
import cv2
import pickle
from tensorflow.keras.utils import to_categorical

with open('name.pickle', 'rb') as f:
    name = pickle.load(f)

In [ ]:
def read_image(link):
    image = cv2.imread(link)
    # convert to GRAY
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return cv2.resize(image, (224, 224), interpolation=cv2.INTER_CUBIC)

# Lấy dữ liệu
def take_data(path):
  # tạo 2 list X và Y
    X = []
    Y = []

    for n in os.listdir(path):
        images = os.path.join(path, n)
        for img in os.listdir(images):
            image = os.path.join(images, img)
            print(image)
            image = read_image(image)
            X.append(image)
            Y.append(name[n])
    # chuyển đổi list thành array.
    X = np.array([i for i in X]).reshape(-1, 224, 224, 1)
    X = X / 255.0
    # chuyển dổi list Y thành array và chuẩn hóa về các giá trị 0,1 trong mảng.
    Y = np.array([i for i in Y])
    Y = to_categorical(Y)
    return X, Y

In [ ]:
# lấy dữ liệu từ folder merger cho tập train
X_train, Y_train = take_data('merger')

In [ ]:
# kiểm tra shape của X_train
X_train.shape

(4992, 224, 224, 1)

In [ ]:
# lấy dữ liệu từ folder vali_1 cho tập vali
X_vali, Y_vali = take_data('vali_1')

In [ ]:
# kiểm tra lại shape X_vali
X_vali.shape

(1094, 224, 224, 1)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, ZeroPadding2D, MaxPooling2D, LeakyReLU, \
    BatchNormalization

def _build_model():
    # CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))


    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(31, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                       optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])
    return model
model = _build_model()

In [ ]:
model.fit(X_train, Y_train, 
          batch_size=32,
          validation_data=(X_vali, Y_vali),
          epochs=1000)

Epoch 1/1000
156/156 [==============================] - 11s 69ms/step - loss: 3.2806 - accuracy: 0.0679 - precision: 0.3143 - recall: 0.0022 - val_loss: 2.4006 - val_accuracy: 0.2367 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/1000
156/156 [==============================] - 10s 65ms/step - loss: 1.8229 - accuracy: 0.3670 - precision: 0.6095 - recall: 0.1979 - val_loss: 1.0477 - val_accuracy: 0.6024 - val_precision: 0.7084 - val_recall: 0.4241
Epoch 3/1000
156/156 [==============================] - 10s 65ms/step - loss: 1.2774 - accuracy: 0.5262 - precision: 0.7233 - recall: 0.3728 - val_loss: 0.6325 - val_accuracy: 0.8080 - val_precision: 0.9104 - val_recall: 0.6782
Epoch 4/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.9733 - accuracy: 0.6462 - precision: 0.7852 - recall: 0.5184 - val_loss: 0.4440 - val_accuracy: 0.8729 - val_precision: 0.9304 - val_recall: 0.7450
Epoch 5/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.

In [ ]:
model.save('model_new_10.h5')

In [ ]:
X_test, Y_test = take_data('test_1')

In [ ]:
len(X_test)

7069

In [ ]:
# kết quả với 7069 ảnh test
model.evaluate(X_test, Y_test)

221/221 [==============================] - 5s 21ms/step - loss: 0.1363 - accuracy: 0.9847 - precision: 0.9869 - recall: 0.9836


[0.13631878793239594,
 0.9847220182418823,
 0.9869410991668701,
 0.983590304851532]